# Setup

In [27]:
source("/home/vickydiliscia/dmeyf2025/src/churn/5002/config.r")

In [2]:
# Cargo el index (que a su vez definirá la función)
source(file.path(PATH_UTILS, "index.R"))

# Ejecuto la función que carga todos los utils
source_utils(PATH_UTILS)

Sourcing: 01_clase_ternaria.R

Sourcing: 02_percentilizar_variables.R

Sourcing: 03_agregar_lags_dataset.R

Sourcing: 04_entrenar_modelos.R

Sourcing: 05_ensamblar_promedio.R

Sourcing: 06_realidad_inicializar.R

Sourcing: 07_realidad_evaluar.R

Sourcing: 08_particionar.R

Sourcing: 09_armar_archivo_envios.R

Sourcing: 10_enviar_kaggle.R

Sourcing: 11_comparar_cortes.R

✅ Todos los utils fueron cargados correctamente.



## Parametros Experimento

In [5]:
## Semillas
PARAM <- list()
PARAM$semilla_primigenia <- SEEDS[1]

# training y future
PARAM$train <- c(202102)
PARAM$train_final <- c(202101,202102)
PARAM$future <- c(202104)
PARAM$future_final <- c(202104)
PARAM$semilla_kaggle <- 314159
PARAM$cortes <- seq(6000, 19000, by= 500)

# undersampling
PARAM$trainingstrategy$undersampling <- 0.1


## Parametros LGBM

In [6]:
PARAM$hyperparametertuning$xval_folds <- 5

# parametros fijos del LightGBM que se pisaran con la parte variable de la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt", # puede ir  dart  , ni pruebe random_forest
  objective= "binary",
  metric= "auc",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE, # para reducir warnings
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_depth= -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split= 0, # min_gain_to_split >= 0
  min_sum_hessian_in_leaf= 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1= 0.0, # lambda_l1 >= 0.0
  lambda_l2= 0.0, # lambda_l2 >= 0.0
  max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  bagging_fraction= 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction= 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction= 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance= FALSE, #
  scale_pos_weight= 1.0, # scale_pos_weight > 0.0

  drop_rate= 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop= 50, # <=0 means no limit
  skip_drop= 0.5, # 0.0 <= skip_drop <= 1.0

  extra_trees= FALSE,

  num_iterations= 1200,
  learning_rate= 0.02,
  feature_fraction= 0.5,
  num_leaves= 750,
  min_data_in_leaf= 5000
)

## Librerias

In [8]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: parallel

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings


Loading required package: primes

Loading required package: rlist

Loading required packa

# Dataset

In [10]:
# lectura del dataset
dataset <- fread(DATASET_FEATURES, stringsAsFactors= TRUE)

In [11]:
# clase01 - me daba error con los max_bin
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

dataset_train <- dataset[foto_mes %in% PARAM$train_final]
dataset_train[,.N,clase_ternaria]

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

clase_ternaria,N
<fct>,<int>
CONTINUA,320372
BAJA+2,1857
BAJA+1,1453


## Tomar Hiperparametros Bayesiana

In [12]:
setwd(PATH_BO)

In [13]:
kbayesiana <- "bayesiana.RDATA"  # tu archivo con el opt.state adentro
# Cargar el archivo (esto mete en memoria los objetos guardados, p. ej. opt.state)
load(kbayesiana)

# Si el archivo contiene el estado de la optimización:
bayesiana_salida <- list(opt.path = opt.state$opt.path)
                         
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

PARAM$out$lgbm$y <- tb_bayesiana[1, y]

print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

[1] "num_iterations"   "learning_rate"    "feature_fraction" "num_leaves"      
 [5] "min_data_in_leaf" "y"                "dob"              "eol"             
 [9] "error.message"    "exec.time"        "ei"               "error.model"     
[13] "train.time"       "prop.type"        "propose.time"     "se"              
[17] "mean"

   num_iterations learning_rate feature_fraction num_leaves min_data_in_leaf
            <int>         <num>            <num>      <int>            <int>
1:           1673     0.0226382        0.3068103        475                1
[1] 0.9437737


# Training

In [17]:
setwd(PATH_THIS_EXPERIMENT)
test_models <- "test_models"
dir.create(test_models, showWarnings= FALSE)
setwd(PATH_TEST_MODELS)

In [18]:
# clase01
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

dataset_train <- dataset[foto_mes %in% PARAM$train_final]
dataset_train[,.N,clase_ternaria]

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train[, campos_buenos, with= FALSE]),
  label= dataset_train[, clase01]
)

clase_ternaria,N
<fct>,<int>
CONTINUA,320372
BAJA+2,1857
BAJA+1,1453


## Setup Hiperparametros

In [20]:
param_final <- modifyList(PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros)

param_final

$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] FALSE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$seed
[1] 100271

$max_depth
[1] -1

$min_gain_to_split
[1] 0

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0

$lambda_l2
[1] 0

$max_bin
[1] 31

$bagging_fraction
[1] 1

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$extra_trees
[1] FALSE

$num_iterations
[1] 1673

$learning_rate
[1] 0.0226382

$feature_fraction
[1] 0.3068103

$num_leaves
[1] 475

$min_data_in_leaf
[1] 1

In [21]:
# este punto es muy SUTIL  y será revisado en la Clase 05

param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)
param_normalizado$num_threads <- 8  # no optimizable, solo controla CPU

In [ ]:
result <- entrenar_modelos(
  dtrain  = dtrain_final,
  params  = param_normalizado,
  nrounds = 1375,
  seeds   = SEEDS,
  tag     = paste0(TAG, "_test")
)
# paths de modelos guardados:
sapply(result, function(x) x$path)

# Ensamblar Prediccion

In [25]:
model_paths <- list.files(pattern = paste0("^modelo_", TAG, "_test", "_seed.*\\.txt$"))
modelos <- lapply(model_paths, lgb.load)

In [28]:
setwd(PATH_THIS_EXPERIMENT)
test_prediction <- "test_prediction"
dir.create(test_prediction, showWarnings= FALSE)
setwd(PATH_TEST_PREDICTION)

In [ ]:
ensamblar_promedio (
  dataset       = dataset,
  PARAM         = PARAM,
  campos_buenos = campos_buenos,
  experimento   = paste0(TAG, "_test"),
  models_dir    = PATH_TEST_MODELS,
  pattern       = paste0("^modelo_", TAG, "_test", "_seed.*\\.txt$"),
)

In [ ]:
# Leer archvio predicciones
setwd(PATH_TEST_PREDICTION)
pred_file <- paste0("prediccion_",TAG,"_test", ".txt")

# Verificá que exista
stopifnot(file.exists(pred_file))

tb_prediccion <- fread(pred_file, sep = "\t")

In [ ]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$future_final]

# inicilizo el dataset  drealidad
drealidad <- realidad_inicializar( dfuture, PARAM)

cortes <- PARAM$cortes 

In [ ]:
comparaciones <- comparar_cortes(tb_prediccion, cortes, drealidad)
plot_cortes(comparaciones)

In [ ]:
res <- comparar_semillas(
  pfuture       = dfuture,
  pparam        = PARAM,          # o tu lista de parámetros
  tb_prediccion = tb_prediccion,  # con columnas numero_de_cliente, prob
  cortes        = PARAM$cortes,   # p.ej. c(8000, 9000, 10000, ...)
  semillas      = SEEDS
)
plot_semillas(res)